La cellule suivante sert à charger les fonctions nécessaires à l'exécution des tests pour les fonctions du TP. N'oubliez donc pas de l'exécuter.

In [1]:
Random.self_init ();;

module Utils = struct
  let fonction_test nom b =
    let () = print_endline (Printf.sprintf "%s : %s" nom (if b then "\027[1m\027[32mOK\027[0m" else "\027[1m\027[31méchec\027[0m"))
    in b

  let rec test_liste comp pretty_print f lst  =    
    let rec loop lst ok total =
      match lst with
      | [] -> ok, total
      | (t,tt)::q ->
        loop q (ok + if fonction_test (pretty_print t) (comp (f t) tt) then 1 else 0) (total + 1) 
    in     
    let ok, total = loop lst 0 0 in
    print_endline (Printf.sprintf "%i/%i tests réussis" ok total)
     
  let rec test_liste_avec_nom comp f lst = 
    let rec loop lst ok total =
      match lst with
      | [] -> ok, total
      | (nom, t, tt)::q ->
        loop q (ok + if fonction_test nom (comp (f t) tt) then 1 else 0) (total + 1) 
    in     
    let ok, total = loop lst 0 0 in
    print_endline (Printf.sprintf "%i/%i tests réussis" ok total)
    
  let uncurry2 f (x,y) = f x y      
  let uncurry3 f (x,y,z) = f x y z

  let print_int_liste l = List.map string_of_int  l |> String.concat "; " |> Printf.sprintf "[%s]"      
  let print_int_array a = Array.to_list a |> List.map string_of_int |> String.concat "; " |> Printf.sprintf "[|%s|]"
      
  let shuffle a =
      let n = Array.length a in
      for i = n - 1 downto 1 do
        let k = Random.int (i+1) in let x = a.(k) in a.(k) <- a.(i); a.(i) <- x
      done
      
  let rec perm_alea n = let t = Array.init n (fun i -> i) in shuffle t;  t  
  
end 

module Test = struct  
  let est_permutation f = 
    let pretty_print a = Printf.sprintf "Tableau %s" (Utils.print_int_array a) in
    let tests = List.init 5 (fun i -> (Utils.perm_alea (i+3), true)) @ [([|0; 2; 0|], false); ([|1; 2; 0; 4|], false)] in
    Utils.test_liste (=) pretty_print f tests
    
  let support f = 
    let pretty_print a = Printf.sprintf "Permutation %s" (Utils.print_int_array a) in
    let tests = [([|0; 1; 2|],[]); ([|2; 1; 0|],[0; 2]); ([|3; 5; 0; 6; 1; 2; 4|],[0; 1; 2; 3; 4; 5; 6]); ([|6; 0; 5; 1; 4; 3; 2|], [0; 1; 2; 3; 5; 6])] in
    Utils.test_liste (fun r rt -> List.sort compare r = rt) pretty_print f tests
    
  let compose f = 
    let pretty_print (a,b) = Printf.sprintf "Permutations %s et %s" (Utils.print_int_array a)  (Utils.print_int_array b) in
    let tests = [(([|3; 5; 0; 6; 1; 2; 4|],[|6; 0; 5; 1; 4; 3; 2|]), [|4; 3; 2; 5; 1; 6; 0|]) ] in
    Utils.test_liste (=) pretty_print (Utils.uncurry2 f) tests
    
  let indice_max_croit f = 
    let pretty_print a = Printf.sprintf "Tableau %s" (Utils.print_int_array a) in
    let tests = [([|2; 3; 0; 1|], 2); ([|2; 3; 1; 0|], 0); ([|0; 1; 2; 3|], 2); ([|0; 1; 4; 3; 2|], 1)] in
    Utils.test_liste (=) pretty_print f tests
    
  let suivant f = 
    let pretty_print a = Printf.sprintf "Permutation %s" (Utils.print_int_array a) in
    let tests = [([|3; 2; 0; 1|], [|3; 2; 1; 0|]); ([|2; 3; 0; 1|], [|2; 3; 1; 0|]); ([|2; 3; 1; 0|], [|3; 0; 1; 2|]); ([|0; 1; 2; 3|], [|0; 1; 3; 2|]); ([|0; 1; 4; 3; 2|], [|0; 2; 1; 3; 4|])] in
    Utils.test_liste (=) pretty_print (fun t -> f t ; t) tests
    
  let compose_cycle f =
    let pretty_print (c, p) = Printf.sprintf "Cycle %s, permutation %s" (Utils.print_int_liste c)  (Utils.print_int_array p) in
    let tests = [(([0; 5 ;1],[|0; 1; 4; 2; 3; 5|]), [|5; 0; 4; 2; 3; 1|])] in
    Utils.test_liste (=) pretty_print (fun (c,p) -> f c p ; p) tests
    
  let produit_cycles f =
    let pretty_print (l,_) = Printf.sprintf "Cycles %s" (List.map Utils.print_int_liste l |> String.concat "; "  |> Printf.sprintf "[%s]")  in
    let tests = [(([[1; 3; 0];[4; 6; 8; 7];[2; 5]],11),[|1; 3; 5; 0; 6; 2; 8; 4; 7; 9; 10|])] in 
    Utils.test_liste (=) pretty_print (Utils.uncurry2 f) tests   
    
  let cycles f =
    let rotate lst =
      let rec aux deb fin reste =
      match reste, deb with
      | [], _ -> deb @ fin
      | _, [] -> failwith "Liste vide"
      | t::q, mini::_ -> if t < mini then aux [t] (fin@deb) q else aux (deb@[t]) fin q
      in
      match lst with
      | [] -> failwith "Liste vide"
      | t::q -> aux [t] [] q
    in  
    let pretty_print a = Printf.sprintf "Permutation %s" (Utils.print_int_array a) in
    let tests = [([|2; 0; 1; 4; 3|], [ [0; 2; 1]; [3; 4]]); ([|1; 3; 5; 0; 6; 2; 8; 4; 7; 9; 10|], [[1; 3; 0];[4; 6; 8; 7];[2; 5]])] in 
    let test_comp r rt = let sort lst = List.sort compare (List.map rotate lst) in sort r = sort rt in
    Utils.test_liste test_comp pretty_print f tests   
end

- : unit = ()


module Utils :
  sig
    val fonction_test : string -> bool -> bool
    val test_liste :
      ('a -> 'b -> bool) ->
      ('c -> string) -> ('c -> 'a) -> ('c * 'b) list -> unit
    val test_liste_avec_nom :
      ('a -> 'b -> bool) -> ('c -> 'a) -> (string * 'c * 'b) list -> unit
    val uncurry2 : ('a -> 'b -> 'c) -> 'a * 'b -> 'c
    val uncurry3 : ('a -> 'b -> 'c -> 'd) -> 'a * 'b * 'c -> 'd
    val print_int_liste : int list -> string
    val print_int_array : int array -> string
    val shuffle : 'a array -> unit
    val perm_alea : int -> int array
  end


module Test :
  sig
    val est_permutation : (int array -> bool) -> unit
    val support : (int array -> int list) -> unit
    val compose : (int array -> int array -> int array) -> unit
    val indice_max_croit : (int array -> int) -> unit
    val suivant : (int array -> 'a) -> unit
    val compose_cycle : (int list -> int array -> 'a) -> unit
    val produit_cycles : (int list list -> int -> int array) -> unit
    val cycles : (int array -> int list list) -> unit
  end


----

Soit $n$ un entier naturel non nul. Une *permutation* $\sigma$ de
l’ensemble $[\![ 0, n-1 ]\!]$ des entiers compris entre $0$
et $n-1$ est une bijection de cet ensemble dans lui-même.

Représentation d’une permutation
================================

Nous représenterons une permutation $\sigma$ de
$[\![ 0, n-1 ]\!]$ par un tableau d’entiers $t$ de longueur
$n$, tel que pour tout $i \in [\![ 0, n-1 ]\!]$, l’entier
$t.(i)$ soit égal à $\sigma(i)$.

<font size="5">❓</font> À quelle condition un tableau $t$ de longueur $n$ représente-t-il une
permutation de $[\![ 0, n-1 ]\!]$ ?

✍️ *Votre réponse*

<font size="5">👩🏽‍💻</font> En déduire une fonction `est_permutation : int array -> bool` qui vérifie si un tableau
d’entiers représente une permutation.

In [ ]:
(*À remplacer par votre code*)
failwith "Code manquant"

**Tests personnels**

In [ ]:
(* Utiliser la cellule pour exécuter quelques appels et vérifier les résultats obtenus *)

**Tests automatiques**

In [ ]:
(* Exécutez cette cellule pour tester votre fonction *)
let () = Test.est_permutation est_permutation;;

Le *support* d’une permutation $\sigma$ est l’ensemble des points non
fixes de $\sigma$, c’est-à-dire l’ensemble des entiers $i$ tels que
$\sigma(i) \neq i$.

<font size="5">👨‍💻</font> Écrire une fonction `support : int array -> int list` qui prend en argument un tableau
représentant une permutation et qui renvoie son support.

In [ ]:
(*À remplacer par votre code*)
failwith "Code manquant"

**Tests personnels**

In [9]:
(* Utiliser la cellule pour exécuter quelques appels et vérifier les résultats obtenus *)

**Tests automatiques**

In [ ]:
(* Exécutez cette cellule pour tester votre fonction *)
let () = Test.support support;;

La composée de deux permutations $\sigma$ et $\sigma'$ de
$[\![ 0, n-1 ]\!]$ est leur composée $\sigma \circ \sigma'$
au sens habituel des applications. Il s’agit d’une permutation de
$[\![ 0, n-1 ]\!]$.

<font size="5">👩🏿‍💻</font> Écrire une fonction `compose : int array -> int array -> int array` qui prend en argument deux tableaux
représentant des permutations et qui renvoie le tableau représentant
leur composée ; `compose t1 t2` renverra le tableau correspondant à la permutation $\sigma_1 \circ \sigma_2$ où $\sigma_1$ est représentée par `t1`et $\sigma_2$ par `t2`.

In [ ]:
(*À remplacer par votre code*)
failwith "Code manquant"

**Tests personnels**

In [ ]:
(* Utiliser la cellule pour exécuter quelques appels et vérifier les résultats obtenus *)

**Tests automatiques**

In [ ]:
(* Exécutez cette cellule pour tester votre fonction *)
let () = Test.compose compose;;

Énumération des permutations
============================

Ordre sur les permutations
--------------------------

On ordonne les permutations de $[\![ 0, n-1 ]\!]$ selon
l’ordre lexicographique. Nous allons écrire une fonction qui étant
donnée une permutation représentée par un tableau $t$ calcule son
successeur immédiat. Pour cela, on applique l’algorithme suivant :

1.  On détermine le plus grand entier $i$ tel que $t.(i) < t.(i+1)$ ;

2.  On permute $t.(i)$ avec le plus petit élément qui lui soit plus
    grand parmi $t.(i+1)$,..., $t.(n-1)$ ;

3.  On trie la portion du tableau située à partir de l’indice $i+1$
    compris.

<font size="5">👨🏻‍💻</font> 
Écrire une fonction `indice_max_croit : 'a array -> int` qui prend en argument un tableau
$t$ et renvoie le plus grand indice $i$ tel que $t.(i) < t.(i+1)$.

In [ ]:
(*À remplacer par votre code*)
failwith "Code manquant"

**Tests personnels**

In [ ]:
(* Utiliser la cellule pour exécuter quelques appels et vérifier les résultats obtenus *)

**Tests automatiques**

In [ ]:
(* Exécutez cette cellule pour tester votre fonction *)
let () = Test.indice_max_croit indice_max_croit;;

<font size="5">👩🏾‍💻</font> 
Écrire une fonction `permute_max : 'a array -> int -> unit` qui prend en argument un tableau $t$
et un entier $i$ et qui permute dans le tableau $t$ l’élément d’indice
$i$ avec le plus petit élément qui lui soit plus grand parmi
$t.(i+1)$,..., $t.(n-1)$ (ou $n$ est la longueur de $t$).

In [ ]:
(*À remplacer par votre code*)
failwith "Code manquant"

**Tests personnels**

In [ ]:
(* Utiliser la cellule pour exécuter quelques appels et vérifier les résultats obtenus *)

<font size="5">👨🏼‍💻</font> 
Écrire une fonction `trie_droite : 'a array -> int -> unit` qui prend en argument un tableau $t$
et un entier $k$ et qui trie en place la portion du tableau située à
partir de l’indice $k$.

In [ ]:
(*À remplacer par votre code*)
failwith "Code manquant"

**Tests personnels**

In [ ]:
(* Utiliser la cellule pour exécuter quelques appels et vérifier les résultats obtenus *)

<font size="5">👩‍💻</font> 
En déduire une fonction `suivant : 'a array -> unit` qui prend en argument un tableau $t$
représentant une permutation et modifie le tableau $t$ en place de
manière à obtenir la permutation suivante.

In [ ]:
(*À remplacer par votre code*)
failwith "Code manquant"

**Tests personnels**

In [ ]:
(* Utiliser la cellule pour exécuter quelques appels et vérifier les résultats obtenus *)

**Tests automatiques**

In [ ]:
(* Exécutez cette cellule pour tester votre fonction *)
let () = Test.suivant suivant;;

Affichage des permutations
--------------------------

<font size="5">👨🏿‍💻</font> 
Écrire une fonction `print_perm : int array -> unit` prenant en argument une permutation et
l’affiche sous la forme de votre choix.

In [ ]:
(*À remplacer par votre code*)
failwith "Code manquant"

**Tests personnels**

In [ ]:
(* Utiliser la cellule pour exécuter quelques appels et vérifier les résultats obtenus *)

<font size="5">👩🏼‍💻</font> 
En déduire une fonction `enumere : int -> unit` qui prend en argument un entier $n$ et
qui affiche toutes les permutations de $[\![ 0, n-1 ]\!]$.

In [ ]:
(*À remplacer par votre code*)
failwith "Code manquant"

**Tests personnels**

In [ ]:
(* Utiliser la cellule pour exécuter quelques appels et vérifier les résultats obtenus *)

<font size="5">❓</font> Combien y en a-t-il ?

✍️ *Votre réponse*

Pour les plus rapides - Cycles
==============================

On appelle *cycle* une permutation $\sigma$ tel qu’il existe $k$
éléments distincts $x_0, \dots,x_{k-1}$ tels que le support de $\sigma$
est $\lbrace x_0, \dots, x_{k-1} \rbrace$, et tels que
$\sigma(x_0) = x_1$, $\sigma(x_1) = x_2$, $\dots$,
$\sigma(x_{k-2}) = x_{k-1}$ et $\sigma(x_{k-1}) = x_0$.

Un tel cycle peut être représenté par la liste $[x_0; \dots; x_{k-1}]$.

On peut démontrer que toute permutation se décompose de manière unique
comme un produit (commutatif) de cycles de supports disjoints. On
représentera un produit de cycles sous la forme d’une liste de listes
d’entiers.

<font size="5">❓</font> 
Décomposer (à la main) la permutation `[|4; 6; 7; 1; 2; 5; 3; 0|]` en un
produit de cycles disjoints.

✍️ *Votre réponse*

<font size="5">👨🏽‍💻</font> 
Écrire une fonction `compose_cycle : int list -> 'a array -> unit` prenant en argument un cycle $c$ et
un tableau $t$ représentant une permutation pour laquelle on suppose que
les éléments du cycle sont des points fixes, et qui modifie en place le
tableau $t$ de sorte que le tableau obtenu représente la composée du
cycle et de la permutation initiale.

In [ ]:
(*À remplacer par votre code*)
failwith "Code manquant"

**Tests personnels**

In [ ]:
(* Utiliser la cellule pour exécuter quelques appels et vérifier les résultats obtenus *)

**Tests automatiques**

In [ ]:
(* Exécutez cette cellule pour tester votre fonction *)
let () = Test.compose_cycle compose_cycle;;

<font size="5">👩🏻‍💻</font> 
En déduire une fonction `produit_cycles : int list list -> int -> int array` qui prend pour argument un
produit de cycles disjoints et l’entier $n$ et qui renvoie la permutation de
$[\![ 0, n-1 ]\!]$ représentée par le produit de cycles.

In [ ]:
(*À remplacer par votre code*)
failwith "Code manquant"

**Tests personnels**

In [ ]:
(* Utiliser la cellule pour exécuter quelques appels et vérifier les résultats obtenus *)

**Tests automatiques**

In [ ]:
(* Exécutez cette cellule pour tester votre fonction *)
let () = Test.produit_cycles produit_cycles;;

<font size="5">👨🏾‍💻</font> 
On souhaite maintenant procéder à l’opération inverse. Écrire une
fonction `cycles` qui prend en argument une permutation et qui décompose
cette permutation en un produit de cycles de supports disjoints.

In [ ]:
(*À remplacer par votre code*)
failwith "Code manquant"

**Tests personnels**

In [ ]:
(* Utiliser la cellule pour exécuter quelques appels et vérifier les résultats obtenus *)

**Tests automatiques**

In [ ]:
(* Exécutez cette cellule pour tester votre fonction *)
let () = Test.cycles cycles;;